In [50]:
import glob
from tqdm import tqdm
import pandas as pd

def distribution_class(df):
    dictionary = {
        "class_names": [],
        "number_images": []
    }
    
    for class_name in set(df.class_name):
        dictionary['class_names'].append(class_name)
        dictionary['number_images'].append(df[df.class_name == class_name].shape[0])
    
    return pd.DataFrame(dictionary)


In [51]:
image_paths = glob.glob("/Users/tuananh/tuananh/domain_calibration/datasets/clipart/*/*.jpg")

dictionary = {
    "image_path": [],
    "class_name": [],
}

for image_path in image_paths:
    dictionary["image_path"].append(image_path)
    dictionary["class_name"].append(image_path.split("/")[-2])

df = pd.DataFrame(dictionary)

dict_labels = {k:index for index, k in enumerate(set(df.class_name))}

distribution_df = distribution_class(df)

In [52]:
df

,image_path,class_name
0,/Users/tuananh/tuananh/domain_calibration/data...,lollipop
1,/Users/tuananh/tuananh/domain_calibration/data...,lollipop
2,/Users/tuananh/tuananh/domain_calibration/data...,lollipop
3,/Users/tuananh/tuananh/domain_calibration/data...,lollipop
4,/Users/tuananh/tuananh/domain_calibration/data...,lollipop
...,...,...
48828,/Users/tuananh/tuananh/domain_calibration/data...,dumbbell
48829,/Users/tuananh/tuananh/domain_calibration/data...,dumbbell
48830,/Users/tuananh/tuananh/domain_calibration/data...,dumbbell
48831,/Users/tuananh/tuananh/domain_calibration/data...,dumbbell


In [59]:
# from sklearn.datasets import make_classification
# from sklearn.model_selection import StratifiedKFold

# kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

# df_X = df.drop("class_name", axis=1)
# df_y = df.class_name

# for train_ix, test_ix in kfold.split(df_X, df_y):
#     train_X, test_X = df_X.iloc[train_ix], df_X.iloc[test_ix]
#     train_y, test_y = df_y.iloc[train_ix], df_y.iloc[test_ix]
#     string_train = ""
#     string_test = ""
#     for class_name in set(df.class_name):
#         train = len(train_y[train_y==class_name])
#         test = len(test_y[test_y==class_name])
#         string_train += "%s=%d, " %(class_name, train)
#         string_test += "%s=%d, " %(class_name, test)
    
#     print(f">Train: {string_train}")
#     print(f"\nTest: {string_test}")
#     print("\n")
#     break

In [66]:
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

df_X = df.drop("class_name", axis=1)
df_y = df.class_name

train_ixs = []
test_ixs = []

with pd.ExcelWriter('../../datasets/data_information.xlsx') as writer:
    k_fold = 0
    for train_ix, test_ix in kfold.split(df_X, df_y):
        train_ixs.append(train_ix)
        test_ixs.append(test_ix)
        
        df_ = df.copy()
        df_["phase"] = "train"
        df_.loc[test_ix, ['phase']] = 'val'
        df_.to_excel(writer, sheet_name=f"k=" + str(k_fold), index=False)
        k_fold += 1